In [2]:
import glob
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import warnings

from dataset import *
from eval import *
from torchvision import models


device = "cuda" if torch.cuda.is_available() else "cpu"

patch_paths_20x = glob.glob("data/train_02/level0/*.jpeg")
patch_paths_10x = glob.glob("data/train_02/level1/*.jpeg")
patch_paths_2_5x = glob.glob("data/train_02/level2/*.jpeg")

In [3]:
warnings.filterwarnings("ignore")

model = models.resnet34(pretrained=True).to(device)
model = nn.Sequential(*list(model.children())[:-1])
model.eval()
pass

Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /home/sha/.cache/torch/hub/checkpoints/resnet34-b627a593.pth


100%|██████████| 83.3M/83.3M [00:02<00:00, 41.1MB/s]


In [4]:
features_20x, coords_20x = extract_all_features(model, patch_paths_20x, device)
features_10x, coords_10x = extract_all_features(model, patch_paths_10x, device)
features_2_5x, coords_2_5x = extract_all_features(model, patch_paths_2_5x, device)

Extracting features: 0it [00:00, ?it/s]
Extracting features: 0it [00:00, ?it/s]
Extracting features: 0it [00:00, ?it/s]


In [5]:
labels_20x = cluster_features(features_20x)
labels_10x = cluster_features(features_10x)
labels_2_5x = cluster_features(features_2_5x)

ValueError: Expected 2D array, got 1D array instead:
array=[].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [6]:
class MLPClassifier(nn.Module):
    def __init__(self, input_size=1000, num_classes=5):
        super(MLPClassifier, self).__init__()
        self.fc1 = nn.Linear(input_size, 512)
        self.fc2 = nn.Linear(512, num_classes)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [7]:
train_loader_20x = create_dataloader(features_20x, labels_20x, coords_20x)
train_loader_10x = create_dataloader(features_10x, labels_10x, coords_10x)
train_loader_2_5x = create_dataloader(features_2_5x, labels_2_5x, coords_2_5x)

NameError: name 'labels_20x' is not defined

In [8]:
model = MLPClassifier(input_size=features_20x.shape[1], num_classes=5).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

train_model(model, train_loader_20x, criterion, optimizer, num_epochs=50, device=device)

IndexError: tuple index out of range

In [9]:
evaluate_model(model, train_loader_20x, device)

NameError: name 'train_loader_20x' is not defined

In [10]:
predicted_labels, patch_coords = get_predictions_and_coords_from_loader(model, train_loader_20x, device)
plot_patches_with_labels(patch_coords, predicted_labels)

NameError: name 'train_loader_20x' is not defined